In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose

def load_and_preprocess_dataset(folder_path):
    """
    Load and preprocess images and masks from the specified folder path.

    Args:
    - folder_path (str): Path to the folder containing image and mask files.

    Returns:
    - images (numpy.ndarray): Preprocessed images.
    - masks (numpy.ndarray): Preprocessed masks.
    """
    # Get list of files in the folder
    file_list = sorted(os.listdir(folder_path))
    
    # Initialize lists to store images and masks
    images = []
    masks = []
    
    # Iterate over files
    for file_name in file_list:
        # Load image
        if file_name.endswith('.png'):
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path)
            # Preprocess images as needed (resize, normalize, etc.)
            image = cv2.resize(image, (image_width, image_height))  # Resize image to match model input shape
            image = image / 255.0  # Normalize image to range [0, 1]
            images.append(image)
        
        # Load mask
        elif file_name.endswith('.tif'):
            mask_path = os.path.join(folder_path, file_name)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            # Preprocess masks as needed (resize, thresholding, etc.)
            mask = cv2.resize(mask, (image_width, image_height))    # Resize mask to match model output shape
            
            # Threshold mask to separate disk and cup areas
            _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)  # Adjust threshold values as needed
            
            masks.append(mask)
    
    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)
    
    return images, masks

def outline_optic_disk(original_image, segmentation_mask):
    """
    Outline the optic disk in the original image based on the segmentation mask.

    Args:
    - original_image (numpy.ndarray): Original fundus image.
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic disk.

    Returns:
    - disk_outline (numpy.ndarray): Image with optic disk outline overlaid.
    """
    # Convert segmentation mask to uint8 datatype
    segmentation_mask = segmentation_mask.astype(np.uint8)

    # Find contours of the optic disk
    contours, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the original image for outlining
    disk_outline = original_image.copy()

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 50:  # Ignore small contours
            # Approximate the contour to reduce the number of points
            approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)

            if cv2.contourArea(approx) > 200:  # Threshold
                cv2.drawContours(disk_outline, [approx], -1, (0, 0, 255), thickness=2)  # Red outline for disk

    return disk_outline

def outline_optic_cup(original_image, segmentation_mask):
    """
    Outline the optic cup in the original image based on the segmentation mask.

    Args:
    - original_image (numpy.ndarray): Original fundus image.
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic cup.

    Returns:
    - cup_outline (numpy.ndarray): Image with optic cup outline overlaid.
    """
    # Threshold the segmentation mask
    _, binary_mask = cv2.threshold(segmentation_mask, 127, 255, cv2.THRESH_BINARY)
    binary_mask = binary_mask.astype(np.uint8)  # Convert mask to uint8 format
    
    # Find contours of the optic cup
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the original image for outlining
    cup_outline = original_image.copy()

    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Ignore small contours
            cv2.drawContours(cup_outline, [contour], -1, (0, 255, 255), thickness=2)  # Yellow outline for cup

    return cup_outline

def calculate_cup_to_disk_ratio(segmentation_mask):
    """
    Calculate the cup-to-disk ratio (CDR) based on the segmentation mask.

    Args:
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic disk and cup.

    Returns:
    - cdr (float): Cup-to-disk ratio.
    """
    # Calculate areas of optic disk and cup
    disk_area = np.sum(segmentation_mask == 128)  # Gray
    cup_area = np.sum(segmentation_mask == 255)  # White
    
    # Check if disk area is zero to avoid division by zero
    if disk_area == 0:
        return float('inf')
    
    # Calculate cup-to-disk ratio
    cdr = cup_area / disk_area
    
    return cdr

def unet_model(input_shape):
    """
    Define U-Net model architecture.

    Args:
    - input_shape (tuple): Shape of input images (height, width, channels).

    Returns:
    - model (tensorflow.keras.models.Model): U-Net model.
    """
    inputs = Input(input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Add more convolutional layers and pooling layers...

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Add more convolutional layers and pooling layers...

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Add more convolutional layers and pooling layers...

    up5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(pool4)
    up5 = concatenate([up5, conv4], axis=3)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)

    # Add more upsampling and convolutional layers...

    up6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv3], axis=3)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)

    # Add more upsampling and convolutional layers...

    up7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv2], axis=3)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    # Add more upsampling and convolutional layers...

    up8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv1], axis=3)
    conv8 = Conv2D(32, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(32, 3, activation='relu', padding='same')(conv8)

    # Add more upsampling and convolutional layers...

    conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv8)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    return model

# Define image dimensions
image_height = 256
image_width = 256

# Example usage
data_folder = r"C:\Users\saira\OneDrive\Desktop\data_folder"
x_train, y_train = load_and_preprocess_dataset(os.path.join(data_folder, "train"))
x_val, y_val = load_and_preprocess_dataset(os.path.join(data_folder, "val"))

# Compile the model
input_shape = (image_height, image_width, 3)
model = unet_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_val, y_val))

# Load and preprocess test dataset
x_test, y_test = load_and_preprocess_dataset(os.path.join(data_folder, "test"))

# Predict on test dataset
predictions = model.predict(x_test)

# Calculate CDR for each prediction and make glaucoma diagnosis
for i in range(len(predictions)):
    optic_disk_outline = outline_optic_disk(x_test[i], predictions[i])
    optic_cup_outline = outline_optic_cup(x_test[i], predictions[i])
    cdr = calculate_cup_to_disk_ratio(predictions[i])
    
    if cdr > 0.5:
        print(f"Image {i+1}: Glaucoma detected. CDR: {cdr:.4f}")
    else:
        print(f"Image {i+1}: No glaucoma detected. CDR: {cdr:.4f}")


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 101s 12s/step - accuracy: 0.0296 - loss: -3.8152 - val_accuracy: 1.5259e-05 - val_loss: -60.6767
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 80s 11s/step - accuracy: 5.2834e-06 - loss: -60.7643 - val_accuracy: 0.0000e+00 - val_loss: -60.6438
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 74s 10s/step - accuracy: 0.0000e+00 - loss: -60.6380 - val_accuracy: 0.0000e+00 - val_loss: -60.6438
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 74s 11s/step - accuracy: 0.0000e+00 - loss: -59.8579 - val_accuracy: 0.0000e+00 - val_loss: -60.7855
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 76s 11s/step - accuracy: 0.0000e+00 - loss: -61.0863 - val_accuracy: 0.0000e+00 - val_loss: -60.6438
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 76s 11s/step - accuracy: 0.0000e+00 - loss: -61.6336 - val_accuracy: 0.0000e+00 - val_loss: -60.6438
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 76s 11s/step - accuracy: 0.0000e+00 - loss: -61.0161 - val_accuracy: 0.0000e+00 - val_loss: -60.6438
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 81s 12s/s

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import jaccard_score  # IoU metric

# Flatten masks and predictions for easier calculation
y_test_flat = y_test.flatten()
predictions_flat = predictions.flatten()

# Threshold predictions to convert to binary masks
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int).flatten()

# Calculate metrics
accuracy = accuracy_score(y_test_flat, predictions_binary)
# Calculate precision for each class
# Calculate precision for each class
precision = precision_score(y_test_flat, predictions_binary, average='macro', zero_division=1)
# Calculate recall for each class
recall = recall_score(y_test_flat, predictions_binary, average='macro', zero_division=1)
iou = jaccard_score(y_test_flat, predictions_binary, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Intersection over Union (IoU):", iou)


Accuracy: 0.0
Precision: 0.6666666666666666
Recall: 0.3333333333333333
Intersection over Union (IoU): 0.0


In [16]:
import os
import cv2
import numpy as np
import time
from sklearn.cluster import KMeans

def load_and_preprocess_dataset(folder_path):
    """
    Load and preprocess images and corresponding masks from the specified folder path.

    Args:
    - folder_path (str): Path to the folder containing image and mask files.

    Returns:
    - images (numpy.ndarray): Preprocessed images.
    - masks (numpy.ndarray): Preprocessed masks.
    """
    file_list = sorted(os.listdir(folder_path))
    images = []
    masks = []
    
    for file_name in file_list:
        if file_name.endswith('.png'):
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (image_width, image_height))
            image = image / 255.0
            images.append(image)
        
        elif file_name.endswith('.tif'):
            mask_path = os.path.join(folder_path, file_name)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (image_width, image_height))
            _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
            masks.append(mask)
    
    images = np.array(images)
    masks = np.array(masks)
    
    return images, masks

def outline_optic_disk(original_image, segmentation_mask):
    """
    Outline the optic disk in the original image based on the segmentation mask.

    Args:
    - original_image (numpy.ndarray): Original fundus image.
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic disk.

    Returns:
    - disk_outline (numpy.ndarray): Image with optic disk outline overlaid.
    """
    segmentation_mask = segmentation_mask.astype(np.uint8)
    contours, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    disk_outline = original_image.copy()

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 50:
            approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)

            if cv2.contourArea(approx) > 200:
                cv2.drawContours(disk_outline, [approx], -1, (0, 0, 255), thickness=2)

    return disk_outline

def outline_optic_cup(original_image, segmentation_mask):
    """
    Outline the optic cup in the original image based on the segmentation mask.

    Args:
    - original_image (numpy.ndarray): Original fundus image.
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic cup.

    Returns:
    - cup_outline (numpy.ndarray): Image with optic cup outline overlaid.
    """
    if segmentation_mask is None:
        return original_image.copy()  # Return a copy of the original image if mask is None
    
    # Ensure segmentation_mask is of type numpy.ndarray
    if not isinstance(segmentation_mask, np.ndarray):
        raise ValueError("segmentation_mask must be a numpy.ndarray")

    # Convert segmentation_mask to uint8 format if necessary
    if segmentation_mask.dtype != np.uint8:
        segmentation_mask = segmentation_mask.astype(np.uint8)
    
    # Threshold the segmentation mask
    _, binary_mask = cv2.threshold(segmentation_mask, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours of the optic cup
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the original image for outlining
    cup_outline = original_image.copy()

    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Ignore small contours
            cv2.drawContours(cup_outline, [contour], -1, (0, 255, 255), thickness=2)  # Yellow outline for cup

    return cup_outline


def calculate_cup_to_disk_ratio(segmentation_mask):
    """
    Calculate the cup-to-disk ratio (CDR) based on the segmentation mask.

    Args:
    - segmentation_mask (numpy.ndarray): Segmentation mask of the optic disk and cup.

    Returns:
    - cdr (float): Cup-to-disk ratio.
    """
    disk_area = np.sum(segmentation_mask == 128)
    cup_area = np.sum(segmentation_mask == 255)
    
    if disk_area == 0:
        return float('inf')
    
    cdr = cup_area / disk_area
    return cdr

def supervised_segmentation(image):
    """
    Perform supervised segmentation using a pre-trained U-Net model.

    Args:
    - image (numpy.ndarray): Input fundus image.

    Returns:
    - segmentation_mask (numpy.ndarray): Segmentation mask.
    """
    start_time = time.time()
    
    # Apply the pre-trained U-Net model for segmentation
    segmentation_mask = model.predict(np.expand_dims(image, axis=0))[0]  # Assuming 'model' is your trained U-Net model
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print("Supervised segmentation time:", elapsed_time, "seconds")
    
    return segmentation_mask

def perform_kmeans(image):
    """
    Perform K-means clustering for unsupervised segmentation.

    Args:
    - image (numpy.ndarray): Input image.

    Returns:
    - segmentation_mask (numpy.ndarray): Segmentation mask.
    """
    # Convert image to 1D array
    image_flat = image.reshape(-1, 3)
    
    # Define the number of clusters (e.g., 2 for foreground/background)
    num_clusters = 2
    
    # Initialize K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    
    # Fit K-means to the image data
    kmeans.fit(image_flat)
    
    # Get cluster centers
    cluster_centers = kmeans.cluster_centers_
    
    # Assign each pixel to the nearest cluster
    labels = kmeans.predict(image_flat)
    
    # Reshape labels to image shape
    segmentation_mask = labels.reshape(image.shape[:2])
    
    return segmentation_mask

def unsupervised_segmentation(image):
    """
    Perform unsupervised segmentation using K-means clustering.

    Args:
    - image (numpy.ndarray): Input image.

    Returns:
    - segmentation_mask (numpy.ndarray): Segmentation mask.
    """
    start_time = time.time()
    
    # Apply K-means clustering for unsupervised segmentation
    segmentation_mask = perform_kmeans(image)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print("Unsupervised segmentation time:", elapsed_time, "seconds")
    
    return segmentation_mask

# Load and preprocess test dataset
data_folder = r"C:\Users\saira\OneDrive\Desktop\data_folder"
x_test, y_test = load_and_preprocess_dataset(os.path.join(data_folder, "test"))

# Define image dimensions
image_height = 256
image_width = 256

# Perform segmentation and evaluation
for i in range(len(x_test)):
    # Unsupervised segmentation
    unsupervised_mask = unsupervised_segmentation(x_test[i])
    unsupervised_od_outline = outline_optic_disk(x_test[i], unsupervised_mask)
    unsupervised_oc_outline = outline_optic_cup(x_test[i], unsupervised_mask)
    unsupervised_cdr = calculate_cup_to_disk_ratio(unsupervised_mask)
    
    # Supervised segmentation
    supervised_mask = supervised_segmentation(x_test[i])
    supervised_od_outline = outline_optic_disk(x_test[i], supervised_mask)
    supervised_oc_outline = outline_optic_cup(x_test[i], supervised_mask)
    supervised_cdr = calculate_cup_to_disk_ratio(supervised_mask)
    
   
    # Print results
    print(f"Image {i+1}:")
    print("Unsupervised Segmentation - CDR:", unsupervised_cdr)
    print("Supervised Segmentation - CDR:", supervised_cdr)
    print()


C:\Users\saira\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Unsupervised segmentation time: 0.8326327800750732 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 951ms/step
Supervised segmentation time: 1.3732318878173828 seconds
Image 1:
Unsupervised Segmentation - CDR: inf
Supervised Segmentation - CDR: inf



C:\Users\saira\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Unsupervised segmentation time: 0.7946202754974365 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Supervised segmentation time: 0.36810755729675293 seconds
Image 2:
Unsupervised Segmentation - CDR: inf
Supervised Segmentation - CDR: inf



C:\Users\saira\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Unsupervised segmentation time: 0.8550002574920654 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Supervised segmentation time: 0.2799561023712158 seconds
Image 98:
Unsupervised Segmentation - CDR: inf
Supervised Segmentation - CDR: inf



C:\Users\saira\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Unsupervised segmentation time: 0.8898665904998779 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Supervised segmentation time: 0.30504560470581055 seconds
Image 99:
Unsupervised Segmentation - CDR: inf
Supervised Segmentation - CDR: inf



C:\Users\saira\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Unsupervised segmentation time: 0.8552303314208984 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
Supervised segmentation time: 0.3065650463104248 seconds
Image 100:
Unsupervised Segmentation - CDR: inf
Supervised Segmentation - CDR: inf



In [7]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog

# Global variables to store resized image and segmentation mask
resize_img = None
resize_seg = None
# Label to display Cup-to-Disk Ratio (CDR)
cdr_label = None

# Function to outline the optic disk in the image based on segmentation mask
def outline_optic_disk(original_image, segmentation_mask):
    # Convert segmentation mask to uint8
    segmentation_mask = segmentation_mask.astype(np.uint8)
    # Find contours of the segmentation mask
    contours, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    disk_outline = original_image.copy()

    # Iterate over contours and draw outlines if conditions met
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 50:
            approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
            if cv2.contourArea(approx) > 200:
                cv2.drawContours(disk_outline, [approx], -1, (0, 0, 255), thickness=2)

    return disk_outline

# Function to outline the optic cup in the image based on segmentation mask
def outline_optic_cup(original_image, segmentation_mask):
    _, binary_mask = cv2.threshold(segmentation_mask, 128, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cup_outline = original_image.copy()

    # Iterate over contours and draw outlines if conditions met
    for contour in contours:
        if cv2.contourArea(contour) > 50:
            cv2.drawContours(cup_outline, [contour], -1, (0, 255, 255), thickness=2)

    return cup_outline

# Function to calculate Cup-to-Disk Ratio (CDR) from segmentation mask
def calculate_cup_to_disk_ratio(segmentation_mask):
    disk_area = np.sum(segmentation_mask == 128)
    cup_area = np.sum(segmentation_mask == 255)
    cdr = cup_area / disk_area if disk_area != 0 else float('inf')
    return cdr

# Function to load images and segmentation masks
def load_images():
    global resize_img, resize_seg
    img_path = filedialog.askopenfilename(title="Select Image")
    seg_path = filedialog.askopenfilename(title="Select Segmentation Mask")
    if img_path and seg_path:  # Check if paths are not empty
        orig_img = cv2.imread(img_path)
        orig_seg = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
        if orig_img is not None and orig_seg is not None:  # Check if images are not None
            resize_img = cv2.resize(orig_img, (800, 600))
            resize_seg = cv2.resize(orig_seg, (800, 600))
        else:
            print("Error: Images could not be loaded. Please check the file paths.")
    else:
        print("Error: No file paths provided.")

# Function to visualize optic disk and cup outlines
def visualize():
    global resize_seg
    disk_outline = outline_optic_disk(resize_img, resize_seg)
    cup_outline = outline_optic_cup(resize_img, resize_seg)
    cv2.imshow("Optic Disk Outline", disk_outline)
    cv2.imshow("Optic Cup Outline", cup_outline)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function to display Cup-to-Disk Ratio (CDR)
def show_results():
    global resize_seg, cdr_label
    cdr_label.config(text="")
    cdr_label.update_idletasks()
    cdr_label.config(text="Cup-to-disk ratio: {:.2f}".format(calculate_cup_to_disk_ratio(resize_seg)))

# Main function to create GUI
def main():
    global resize_img, resize_seg, cdr_label
    root = tk.Tk()
    root.title("Automated Glaucoma Detection")
    root.configure(bg='#F5DEB3')  # Set background color to brown

    # Label for heading
    heading = tk.Label(root, text="Automated Glaucoma Detection", font=("Helvetica", 16), bg='#F5DEB3')  # Set background color to brown
    heading.pack(pady=10)

    # Button to load images
    load_button = tk.Button(root, text="Load Images", command=load_images, bg='#8B4513', fg='white', font=("Helvetica", 12), relief=tk.RAISED, padx=20, pady=10, bd=5)  # Set background color to brown
    load_button.pack(pady=10)

    # Button to visualize optic disk and cup outlines
    visualize_button = tk.Button(root, text="Visualize", command=visualize, bg='#8B4513', fg='white', font=("Helvetica", 12), relief=tk.RAISED, padx=20, pady=10, bd=5)  # Set background color to brown
    visualize_button.pack(pady=10)

    # Button to calculate and display Cup-to-Disk Ratio (CDR)
    calculate_button = tk.Button(root, text="Calculate CDR", command=show_results, bg='#8B4513', fg='white', font=("Helvetica", 12), relief=tk.RAISED, padx=20, pady=10, bd=5)  # Set background color to brown
    calculate_button.pack(pady=10)

    # Label to display Cup-to-Disk Ratio (CDR)
    cdr_label = tk.Label(root, text="", bg='#F5DEB3', font=("Helvetica", 12))  # Set background color to brown
    cdr_label.pack(pady=10)

    # Spacer for layout
    spacer = tk.Label(root, text="", bg='#F5DEB3')
    spacer.pack()

    root.geometry("1000x550")
    root.mainloop()

if _name_ == "_main_":
    main()